In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
train_df = pd.read_csv('/data/kaggle/plant-pathology-2021-fgvc8/train.csv', index_col='image')
train_files = os.listdir('/data/kaggle/plant-pathology-2021-fgvc8/train_images')

In [3]:
len(train_df), len(train_files)

(18632, 18632)

In [4]:
with open('./duplicates.csv', 'r') as file:
    duplicates = [x.strip().split(',') for x in file.readlines()]
init_len = len(train_df)
for row in duplicates:
    unique_labels = train_df.loc[row].drop_duplicates().values
    if len(unique_labels) == 1:
        train_df = train_df.drop(row[1:], axis=0)
    else:
        train_df = train_df.drop(row, axis=0)
        
print(f'Dropping {init_len - len(train_df)} duplicate samples.')

Dropping 83 duplicate samples.


In [5]:
labels = train_df['labels'].to_list()

In [6]:
import collections
c = collections.Counter(labels)
c

Counter({'healthy': 4624,
         'scab frog_eye_leaf_spot complex': 200,
         'scab': 4818,
         'complex': 1580,
         'rust': 1815,
         'frog_eye_leaf_spot': 3180,
         'powdery_mildew': 1184,
         'scab frog_eye_leaf_spot': 686,
         'frog_eye_leaf_spot complex': 165,
         'rust frog_eye_leaf_spot': 119,
         'powdery_mildew complex': 87,
         'rust complex': 91})

In [7]:
train_df.head()

,labels
image,
800113bb65efe69e.jpg,healthy
8002cb321f8bfcdf.jpg,scab frog_eye_leaf_spot complex
80070f7fb5e2ccaa.jpg,scab
80077517781fb94f.jpg,scab
800cbf0ff87721f8.jpg,complex


In [8]:
from sklearn.preprocessing import MultiLabelBinarizer

In [9]:
train_df['labels'] = [x.split(' ') for x in train_df['labels']]

In [10]:
train_df.head()

,labels
image,
800113bb65efe69e.jpg,[healthy]
8002cb321f8bfcdf.jpg,"[scab, frog_eye_leaf_spot, complex]"
80070f7fb5e2ccaa.jpg,[scab]
80077517781fb94f.jpg,[scab]
800cbf0ff87721f8.jpg,[complex]


In [11]:
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(train_df['labels'].values)

In [12]:
labels

array([[0, 0, 1, 0, 0, 0],
       [1, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       ...,
       [0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0]])

In [13]:
mlb.classes_

array(['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew',
       'rust', 'scab'], dtype=object)

In [14]:
new_train_df = pd.DataFrame(columns=mlb.classes_, data=labels)

In [15]:
new_train_df.insert(0,'image',train_df.index)

In [16]:
new_train_df.head()

,image,complex,frog_eye_leaf_spot,healthy,powdery_mildew,rust,scab
0,800113bb65efe69e.jpg,0,0,1,0,0,0
1,8002cb321f8bfcdf.jpg,1,1,0,0,0,1
2,80070f7fb5e2ccaa.jpg,0,0,0,0,0,1
3,80077517781fb94f.jpg,0,0,0,0,0,1
4,800cbf0ff87721f8.jpg,1,0,0,0,0,0


In [17]:
import torch
from torch import nn
from torch import optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torchvision import utils
import albumentations as A
import albumentations.pytorch
#from torchsummary import summary
import cv2
import os
import copy
import numpy as np


In [18]:
from torchvision import transforms
from PIL import Image

#use albumentations augmentation
class PlantPathologyDataset(Dataset):
    def __init__(self, root_path, X, y, transform=None):
        self.root_path = root_path

        self.X = X
        self.y = y
        self.transform = transform
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):

        img_path = self.X[index]
        label = self.y[index]
        #image = Image.open(os.path.join(self.root_path,img_path))
        image = cv2.imread(os.path.join(self.root_path,img_path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform:
            augmentated = self.transform(image=image)
            image = augmentated['image']
        
        return image, label   

In [19]:
X,Y = new_train_df['image'].to_numpy(), new_train_df[["healthy", "scab", "frog_eye_leaf_spot", "complex","rust","powdery_mildew"]].to_numpy(dtype=np.float32)

In [20]:
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

msss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=1234)

for train_index, test_index in msss.split(X, Y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]

TRAIN: [    0     1     2 ... 18544 18546 18548] TEST: [    3    13    15 ... 18540 18545 18547]


/home/jhlee/anaconda3/envs/kaggle_plant/lib/python3.7/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass test_size=0.2, train_size=None, random_state=1234 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [21]:
import time
import copy
from tqdm import tqdm
from apex import amp, optimizers
# TRAIN
def train_model(datasets, dataloaders, model, criterion, optimizer, scheduler, num_epochs, device):
    since = time.time()
    model = model.to(device)
    #amp
    opt_level = 'O1'
    model, optimizer = amp.initialize(model, optimizer, opt_level=opt_level)
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1111111111111111111.
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs-1))
        print('-' * 10)
        
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0.0
            
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
               
                # Zero out the grads
                optimizer.zero_grad()
                
                # Forward
                # Track history in train mode
                #with torch.set_grad_enabled(phase == 'train'):
                #model = model.to(device)
                outputs = model(inputs)
                #_, preds = torch.max(outputs, 1) 
                loss = criterion(outputs, labels)

                if phase == 'train':
                    
                    #loss.backward()
                    with amp.scale_loss(loss, optimizer) as scaled_loss:
                        scaled_loss.backward()
                    optimizer.step()
                
                # Statistics
                running_loss += loss.item()*inputs.size(0)
                #running_corrects += torch.sum(preds == labels.data)
            
            if phase == 'valid':
                scheduler.step(running_loss)
                
            epoch_loss = running_loss / len(datasets[phase])
            #epoch_acc = running_corrects.double() / len(datasets[phase])
            
            print('{} Loss: {:.4f} '.format(
                phase, epoch_loss))
            
            if phase == 'valid' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(best_model_wts, '/data/kaggle/plant-pathology-2021-fgvc8/epoch{}_efficientnet-b0.pth'.format(epoch))
        
        print()
    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Loss: {:.4f}'.format(best_loss))
    
    model.load_state_dict(best_model_wts)
    
    return model

In [22]:
import torch.nn.functional as F


class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, logits=False, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduce = reduce

    def forward(self, inputs, targets):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets)
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss

In [23]:
A_train_transform = A.Compose([
    A.Resize(224,224), 
    
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(p=0.5),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=10, val_shift_limit=10, p=0.7),
    A.RandomBrightnessContrast(brightness_limit=(-0.2,0.2), contrast_limit=(-0.2, 0.2), p=0.7),
    A.CLAHE(clip_limit=(1,4), p=0.5),
    A.OneOf([
       A.OpticalDistortion(distort_limit=1.0),
       A.GridDistortion(num_steps=5, distort_limit=1.),
       A.ElasticTransform(alpha=3),
    ], p=0.2),   
    A.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),    
    A.pytorch.transforms.ToTensor()
])

A_test_transform = A.Compose([
    A.Resize(224,224), 
    A.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),    
    A.pytorch.transforms.ToTensor()
])

In [24]:
from efficientnet_pytorch import EfficientNet

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
train_root_path = '/data/kaggle/plant-pathology-2021-fgvc8/train_images/'

train_dataset = PlantPathologyDataset(root_path = train_root_path, X=X_train, y=y_train, transform=A_train_transform)
train_loader = DataLoader(train_dataset, batch_size=64, num_workers=4, shuffle=True, pin_memory=True)
val_dataset = PlantPathologyDataset(root_path = train_root_path, X=X_test, y=y_test, transform=A_test_transform)
val_loader = DataLoader(val_dataset, batch_size=16, num_workers=4, shuffle=False, pin_memory=True)

datasets = {'train': train_dataset,
            'valid': val_dataset}

dataloaders = {'train': train_loader,
               'valid': val_loader}

# LOAD PRETRAINED ViT MODEL
model = EfficientNet.from_pretrained('efficientnet-b1', num_classes=6)

# OPTIMIZER
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.001)
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=0.001)
# optimizer = AdamP(model.parameters(), lr=1e-4, weight_decay=0.001)

# LEARNING RATE SCHEDULER
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=1, factor=0.1)

criterion = FocalLoss(logits=True)
num_epochs = 30

cuda
Loaded pretrained weights for efficientnet-b1


In [25]:
trained_model = train_model(datasets, dataloaders, model, criterion, optimizer, scheduler, 20, device)

  0%|          | 0/233 [00:00<?, ?it/s]

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Epoch 0/19
----------


  0%|          | 0/231 [00:00<?, ?it/s]

train Loss: 0.0400 


  0%|          | 0/233 [00:00<?, ?it/s]

...

train Loss: 0.0009 


  0%|          | 0/233 [00:00<?, ?it/s]

valid Loss: 0.0014 

Epoch 16/19
----------


  3%|▎         | 8/233 [00:17<08:01,  2.14s/it]


KeyboardInterrupt: 